In [1]:
import pandas as pd


In [2]:
url14 = 'http://www.sos.state.mn.us/Modules/ShowDocument.aspx?documentid=14542'
e2014 = pd.read_excel(url14)
e2014['Year'] = '2014'
e2014.shape[0]


4106

In [3]:
e2014.head()

,VTDID,PCTNAME,PCTCODE,MCDNAME,COUNTYNAME,COUNTYCODE,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,...,MNAUDTOTAL,MNAGIP,MNAGR,MNAGDFL,MNAGGP,MNAGLIB,MNAGLMN,MNAGWI,MNAGTOTAL,Year
0,270010005,Aitkin,5,Aitkin,Aitkin,1,8,10,10B,1,...,751,16,262,426,5,4,35,0,748,2014
1,270010010,Aitkin Twp,10,Aitkin Twp,Aitkin,1,8,10,10B,1,...,424,12,208,184,2,7,9,0,422,2014
2,270010015,Ball Bluff Twp,15,Ball Bluff Twp,Aitkin,1,8,10,10B,5,...,141,2,46,87,0,0,6,0,141,2014
3,270010020,Balsam Twp,20,Balsam Twp,Aitkin,1,8,10,10B,5,...,15,0,6,12,0,0,0,0,18,2014
4,270010025,Beaver Twp,25,Beaver Twp,Aitkin,1,8,10,10B,3,...,36,3,4,26,0,0,3,0,36,2014


In [4]:
url12 = 'http://www.sos.state.mn.us/Modules/ShowDocument.aspx?documentid=12177'
e2012 = pd.read_excel(url12)
e2012['Year'] = '2012'
e2012.shape[0]


4103

In [5]:
e2012.head()

,PCTNAME,PCTCODE,MCDNAME,COUNTYNAME,COUNTYCODE,CONGDIST,MNSENDIST,MNLEGDIST,CTYCOMDIST,JUDDIST,...,MNCA1YES,MNCA1NO,MNCA1EST,MNCA1TOTAL,MNCA2YES,MNCA2NO,MNCA2EST,MNCA2TOTAL,VTDID,Year
0,AITKIN,5,Aitkin,Aitkin,1,8,10,10B,1,9,...,550,414,22,986,477,484,25,986,270010005,2012
1,AITKIN TWP,10,Aitkin Twp,Aitkin,1,8,10,10B,1,9,...,355,151,7,513,296,206,11,513,270010010,2012
2,BALL BLUFF TWP,15,Ball Bluff Twp,Aitkin,1,8,10,10B,5,9,...,111,61,2,174,79,94,1,174,270010015,2012
3,BALSAM TWP,20,Balsam Twp,Aitkin,1,8,10,10B,5,9,...,11,12,0,23,10,13,0,23,270010020,2012
4,BEAVER TWP,25,Beaver Twp,Aitkin,1,8,10,10B,3,9,...,16,27,0,43,17,26,0,43,270010025,2012


In [6]:
url10 = 'http://www.sos.state.mn.us/Modules/ShowDocument.aspx?documentid=9737'
e2010 = pd.read_excel(url10)
e2010['Year'] = '2010'
e2010.shape[0]

4137

At some point, we'll need to figure out which precints are missing between each of the spreadsheets.

In [7]:
e2010.head()

,Precinct Name,Precinct Code,CG,LEG,CM,JD,SW,FIPS,CountyID,County,...,STAUDGR,STAUDWI,STAUDTOT,ATGENIP,ATGENR,ATGENDFL,ATGENTRP,ATGENWI,ATGENTOT,Year
0,AITKIN,5,8,03B,1,9,NaN,460,1,AITKIN,...,12,1,725,47,279,388,9,1,724,2010
1,AITKIN TWP,10,8,03B,1,9,NaN,478,1,AITKIN,...,9,0,390,22,193,174,3,0,392,2010
2,BALL BLUFF TWP,15,8,03B,5,9,NaN,3358,1,AITKIN,...,1,1,141,4,44,93,0,1,142,2010
3,BALSAM TWP,20,8,03B,5,9,NaN,3412,1,AITKIN,...,0,0,22,1,6,14,0,0,21,2010
4,BEAVER TWP,25,8,03B,3,9,NaN,4384,1,AITKIN,...,0,0,31,2,7,24,0,0,33,2010


Unsurprisingl, there is a different number of columns since the same offices were not up every year.  We have a couple of options from this point
1) We can reorder and rename the columns to get them to line up before concatenating the files.
2) We can extract aggregate to district level and then find statistics (DPI, Dem Base, etc) from each year and combine those into a file for further use.
3) ???

Let's go with option 2, it is much easier and we can create some functions to pass that will speed this up.

Before we do that, we need to trim down to columns we want to actually use in the analysis.  For this example, we're using State House, so that will be the only district sorting information we'll keep.  Aditionally, there are a lot of races and ballot initiatives in some of the years that we will skip.  We will also narrow to just DFL seats for the calculations.




In [8]:
#recode the legislative districts for 01 to be 1, etc.
districts = ['01A', '01B', '02A', '02B', '03A', '03B', '04A', '04B', '05A', '05B', '06A', '06B', '07A', '07B', '08A', '08B', '09A', '09B']

def fix_dists(x):
    if x in districts:
        return x[1:]
    else:
        return x
    
e2010['District'] = e2010['LEG'].apply(fix_dists)
e2010.head()


,Precinct Name,Precinct Code,CG,LEG,CM,JD,SW,FIPS,CountyID,County,...,STAUDWI,STAUDTOT,ATGENIP,ATGENR,ATGENDFL,ATGENTRP,ATGENWI,ATGENTOT,Year,District
0,AITKIN,5,8,03B,1,9,NaN,460,1,AITKIN,...,1,725,47,279,388,9,1,724,2010,3B
1,AITKIN TWP,10,8,03B,1,9,NaN,478,1,AITKIN,...,0,390,22,193,174,3,0,392,2010,3B
2,BALL BLUFF TWP,15,8,03B,5,9,NaN,3358,1,AITKIN,...,1,141,4,44,93,0,1,142,2010,3B
3,BALSAM TWP,20,8,03B,5,9,NaN,3412,1,AITKIN,...,0,22,1,6,14,0,0,21,2010,3B
4,BEAVER TWP,25,8,03B,3,9,NaN,4384,1,AITKIN,...,0,31,2,7,24,0,0,33,2010,3B


In [9]:
e2010.columns

Index(['Precinct Name', 'Precinct Code', 'CG', 'LEG', 'CM', 'JD', 'SW', 'FIPS', 'CountyID', 'County', 'VSCode', 'EquipmentModel', 'MailBallot', '7AM', 'EDR', 'Signatures', 'RegMilAB', 'FedAB', 'PresAB', 'TotVoters', 'CONGIP', 'CONGR', 'CONGDFL', 'CONGWI', 'CONGTOT', 'MNSENIP', 'MNSENR', 'MNSENDFL', 'MNSENWI', 'MNSENTOT', 'MNLEGIP', 'MNLEGR', 'MNLEGDFL', 'MNLEGWI', 'MNLEGTOT', 'GOVIP', 'GOVR', 'GOVDFL', 'GOVGP', 'GOVTRP', 'GOVGR', 'GOVEDP', 'GOVWI', 'GOVTOT', 'SOSIP', 'SOSR', 'SOSDFL', 'SOSWI', 'SOSTOT', 'STAUDR', 'STAUDDFL', 'STAUDGP', 'STAUDGR', 'STAUDWI', 'STAUDTOT', 'ATGENIP', 'ATGENR', 'ATGENDFL', 'ATGENTRP', 'ATGENWI', 'ATGENTOT', 'Year', 'District'], dtype='object')

In [10]:
cols_2010 = ['TotVoters', 'CONGDFL', 'CONGTOT', 'MNSENDFL', 'MNSENTOT', 'MNLEGDFL', 'MNLEGTOT', 'GOVDFL', 'GOVTOT', 'SOSDFL', 'SOSTOT', 
             'STAUDDFL', 'STAUDTOT', 'ATGENDFL', 'ATGENTOT', 'District']
e10 = e2010[cols_2010]
e10 = e10.groupby('District', as_index = False).sum()
e10.head()

,District,TotVoters,CONGDFL,CONGTOT,MNSENDFL,MNSENTOT,MNLEGDFL,MNLEGTOT,GOVDFL,GOVTOT,SOSDFL,SOSTOT,STAUDDFL,STAUDTOT,ATGENDFL,ATGENTOT
0,10A,15574,7809,15372,6655,15337,5449,15283,5567,15399,5570,14829,5416,14996,6050,14990
1,10B,16015,7235,15739,7385,15780,5258,15645,5374,15767,5518,15194,5397,15313,6096,15271
2,11A,16606,8851,16275,5968,16356,5496,16398,6402,16443,6986,16018,6715,15903,7810,15899
3,11B,15962,7649,15697,5203,15650,5147,15654,5591,15819,6116,15296,6059,15369,6994,15331
4,12A,16676,6633,16492,6327,16425,9435,16404,6671,16511,6952,16091,7023,15951,7667,15963


In [11]:

e10['USREPPERC'] = e10['CONGDFL']/e10['CONGTOT']
e10['MNLEGPERC'] = e10['MNLEGDFL']/e10['MNLEGTOT']
e10['MNGOVPERC'] = e10['GOVDFL']/e10['GOVTOT']
e10['MNSOSPERC'] = e10['SOSDFL']/e10['SOSTOT']
e10['MNAUDPERC'] = e10['STAUDDFL']/e10['STAUDTOT']
e10['MNAGPERC'] = e10['ATGENDFL']/e10['ATGENTOT']
e10['MNSENPERC'] = e10['MNSENDFL']/e10['MNSENTOT']

dfl_totals10 = ['CONGDFL', 'MNLEGDFL', 'GOVDFL', 'SOSDFL', 'STAUDDFL', 'ATGENDFL', 'MNSENDFL']
dfl_percs10 = ['MNSENPERC', 'USREPPERC', 'MNLEGPERC', 'MNGOVPERC', 'MNSOSPERC', 'MNAUDPERC', 'MNAGPERC']

e10['DEMBASE_CNT'] = e10[dfl_totals10].min(axis = 1)
e10['DEMBASE_PCT'] = e10[dfl_percs10].min(axis = 1)
e10['DEMAV_CNT'] = e10[dfl_totals10].min(axis = 1)
e10['DEMAV_PCT'] = e10[dfl_percs10].min(axis = 1)
e10['Year'] = '2010'
e10.head()

#extra_str = '_extra'
#df.columns = [col + extra_str for col in df.columns]

,District,TotVoters,CONGDFL,CONGTOT,MNSENDFL,MNSENTOT,MNLEGDFL,MNLEGTOT,GOVDFL,GOVTOT,...,MNGOVPERC,MNSOSPERC,MNAUDPERC,MNAGPERC,MNSENPERC,DEMBASE_CNT,DEMBASE_PCT,DEMAV_CNT,DEMAV_PCT,Year
0,10A,15574,7809,15372,6655,15337,5449,15283,5567,15399,...,0.361517,0.375615,0.361163,0.403602,0.433918,5416,0.356540,5416,0.356540,2010
1,10B,16015,7235,15739,7385,15780,5258,15645,5374,15767,...,0.340838,0.363170,0.352446,0.399188,0.467997,5258,0.336082,5258,0.336082,2010
2,11A,16606,8851,16275,5968,16356,5496,16398,6402,16443,...,0.389345,0.436134,0.422247,0.491226,0.364881,5496,0.335163,5496,0.335163,2010
3,11B,15962,7649,15697,5203,15650,5147,15654,5591,15819,...,0.353436,0.399843,0.394235,0.456200,0.332460,5147,0.328798,5147,0.328798,2010
4,12A,16676,6633,16492,6327,16425,9435,16404,6671,16511,...,0.404034,0.432043,0.440286,0.480298,0.385205,6327,0.385205,6327,0.385205,2010


In [12]:
e2014.columns

Index(['VTDID', 'PCTNAME', 'PCTCODE', 'MCDNAME', 'COUNTYNAME', 'COUNTYCODE', 'CONGDIST', 'MNSENDIST', 'MNLEGDIST', 'CTYCOMDIST', 'JUDDIST', 'SWCDIST', 'WARD', 'HOSPDIST', 'PARKDIST', 'TABSYSTEM', 'TABMODEL', 'MAILBALLOT', 'REG7AM', 'EDR', 'SIGNATURES', 'AB_MB', 'FEDONLYAB', 'TOTVOTING', 'USSENIP', 'USSENR', 'USSENDFL', 'USSENLIB', 'USSENWI', 'USSENTOTAL', 'USREPIP', 'USREPR', 'USREPDFL', 'USREPWI', 'USREPTOTAL', 'MNLEGIP', 'MNLEGR', 'MNLEGDFL', 'MNLEGWI', 'MNLEGTOTAL', 'MNGOVIP', 'MNGOVR', 'MNGOVDFL', 'MNGOVLIB', 'MNGOVGLC', 'MNGOVWI', 'MNGOVTOTAL', 'MNSOSIP', 'MNSOSR', 'MNSOSDFL', 'MNSOSLIB', 'MNSOSWI', 'MNSOSTOTAL', 'MNAUDIP', 'MNAUDR', 'MNAUDDFL', 'MNAUDLIB', 'MNAUDGLC', 'MNAUDWI', 'MNAUDTOTAL', 'MNAGIP', 'MNAGR', 'MNAGDFL', 'MNAGGP', 'MNAGLIB', 'MNAGLMN', 'MNAGWI', 'MNAGTOTAL', 'Year'], dtype='object')

In [13]:
cols2014 = ['MNLEGDIST', 'TOTVOTING', 'USSENDFL', 'USSENTOTAL', 'USREPDFL', 'USREPTOTAL', 'MNLEGDFL', 'MNLEGTOTAL', 'MNGOVDFL', 'MNGOVTOTAL', 
            'MNSOSDFL', 'MNSOSTOTAL', 'MNAUDDFL', 'MNAUDTOTAL','MNAGDFL', 'MNAGTOTAL']
e2014 = e2014[cols2014]
e14 = e2014.groupby('MNLEGDIST', as_index = False).sum()
e14.head()


,MNLEGDIST,TOTVOTING,USSENDFL,USSENTOTAL,USREPDFL,USREPTOTAL,MNLEGDFL,MNLEGTOTAL,MNGOVDFL,MNGOVTOTAL,MNSOSDFL,MNSOSTOTAL,MNAUDDFL,MNAUDTOTAL,MNAGDFL,MNAGTOTAL
0,10A,16357,7197,16273,6294,16236,7539,16200,6727,16222,5869,15839,6911,15846,7073,15904
1,10B,18015,8640,17899,7705,17842,8523,17757,8096,17820,7246,17382,8543,17425,8946,17453
2,11A,15575,9658,15474,8275,15455,9493,15226,9127,15413,8555,15109,8826,15099,9129,15158
3,11B,14306,7061,14181,6501,14059,6488,14058,6493,14136,5992,13749,6936,13841,7331,13884
4,12A,17117,8709,16946,8864,16880,8065,16828,7964,16849,6980,16338,7848,16357,8072,16377


In [14]:
e14['USSENPERC'] = e14['USSENDFL']/e14['USSENTOTAL']
e14['USREPPERC'] = e14['USREPDFL']/e14['USREPTOTAL']
e14['MNLEGPERC'] = e14['MNLEGDFL']/e14['MNLEGTOTAL']
e14['MNGOVPERC'] = e14['MNGOVDFL']/e14['MNGOVTOTAL']
e14['MNSOSPERC'] = e14['MNSOSDFL']/e14['MNSOSTOTAL']
e14['MNAUDPERC'] = e14['MNAUDDFL']/e14['MNAUDTOTAL']
e14['MNAGPERC'] = e14['MNAGDFL']/e14['MNAGTOTAL']

In [15]:
dfl_totals14 = ['USSENDFL', 'USREPDFL', 'MNLEGDFL', 'MNGOVDFL', 'MNSOSDFL', 'MNAUDDFL', 'MNAGDFL']
dfl_percs14 = ['USSENPERC', 'USREPPERC', 'MNLEGPERC', 'MNGOVPERC', 'MNSOSPERC', 'MNAUDPERC', 'MNAGPERC']
min1 = lambda x: x.min()
avg1 = lambda x: x.mean()
e14['DEMBASE_CNT'] = e14[dfl_totals14].apply(min1, axis = 1)
e14['DEMBASE_PCT'] = e14[dfl_percs14].apply(min1, axis = 1)
e14['DEMAV_CNT'] = e14[dfl_totals14].apply(avg1, axis = 1)
e14['DEMAV_PCT'] = e14[dfl_percs14].apply(avg1, axis = 1)
e14['Year'] = '2014'
e14.head()

,MNLEGDIST,TOTVOTING,USSENDFL,USSENTOTAL,USREPDFL,USREPTOTAL,MNLEGDFL,MNLEGTOTAL,MNGOVDFL,MNGOVTOTAL,...,MNLEGPERC,MNGOVPERC,MNSOSPERC,MNAUDPERC,MNAGPERC,DEMBASE_CNT,DEMBASE_PCT,DEMAV_CNT,DEMAV_PCT,Year
0,10A,16357,7197,16273,6294,16236,7539,16200,6727,16222,...,0.465370,0.414684,0.370541,0.436135,0.444731,5869,0.370541,6801.428571,0.423055,2014
1,10B,18015,8640,17899,7705,17842,8523,17757,8096,17820,...,0.479980,0.454321,0.416868,0.490273,0.512577,7246,0.416868,8242.714286,0.466939,2014
2,11A,15575,9658,15474,8275,15455,9493,15226,9127,15413,...,0.623473,0.592162,0.566219,0.584542,0.602256,8275,0.535425,9009.000000,0.589746,2014
3,11B,14306,7061,14181,6501,14059,6488,14058,6493,14136,...,0.461517,0.459324,0.435814,0.501120,0.528018,5992,0.435814,6686.000000,0.478017,2014
4,12A,17117,8709,16946,8864,16880,8065,16828,7964,16849,...,0.479261,0.472669,0.427225,0.479795,0.492886,6980,0.427225,8071.714286,0.484412,2014


In [16]:
e2012.columns

Index(['PCTNAME', 'PCTCODE', 'MCDNAME', 'COUNTYNAME', 'COUNTYCODE', 'CONGDIST', 'MNSENDIST', 'MNLEGDIST', 'CTYCOMDIST', 'JUDDIST', 'SWCDIST', 'PARKDIST', 'HOSPDIST', 'WARD', 'TABSYSTEM', 'TABMODEL', 'MAILBALLOT', '7AM', 'EDR', 'SIGNATURES', 'REGMILOVAB', 'FEDONLYAB', 'PRESONLYAB', 'TOTVOTING', 'USPRSR', 'USPRSDFL', 'USPRSLIB', 'USPRSSWP', 'USPRSCP', 'USPRSCG', 'USPRSGP', 'USPRSGR', 'USPRSSL', 'USPRSJP', 'USPRSWI', 'USPRSTOTAL', 'USSENIP', 'USSENR', 'USSENDFL', 'USSENGR', 'USSENMOP', 'USSENWI', 'USSENTOTAL', 'USREPIP', 'USREPR', 'USREPDFL', 'USREPWI', 'USREPTOTAL', 'MNSENIP', 'MNSENR', 'MNSENDFL', 'MNSENWI', 'MNSENTOTAL', 'MNLEGIP', 'MNLEGR', 'MNLEGDFL', 'MNLEGWI', 'MNLEGTOTAL', 'MNCA1YES', 'MNCA1NO', 'MNCA1EST', 'MNCA1TOTAL', 'MNCA2YES', 'MNCA2NO', 'MNCA2EST', 'MNCA2TOTAL', 'VTDID', 'Year'], dtype='object')

In [17]:
cols2012 = ['MNLEGDIST', 'TOTVOTING','USPRSDFL', 'USPRSTOTAL', 'USSENDFL','MNSENDFL', 'MNSENTOTAL', 'USSENTOTAL', 'USREPDFL', 'USREPTOTAL', 'MNLEGDFL', 'MNLEGTOTAL']
e2012 = e2012[cols2012]
e12 = e2012.groupby('MNLEGDIST', as_index = False).sum()

In [18]:
e12['USSENPERC'] = e12['USSENDFL']/e12['USSENTOTAL']
e12['USREPPERC'] = e12['USREPDFL']/e12['USREPTOTAL']
e12['MNLEGPERC'] = e12['MNLEGDFL']/e12['MNLEGTOTAL']
e12['USPRSPERC'] = e12['USPRSDFL']/e12['USPRSTOTAL']
e12['MNSENPERC'] = e12['MNSENDFL']/e12['MNSENTOTAL']

dfl_totals12 = ['USSENDFL', 'USREPDFL', 'MNLEGDFL', 'USPRSDFL', 'MNSENDFL']
dfl_percs12 = ['USSENPERC', 'USREPPERC', 'MNLEGPERC', 'USPRSPERC', 'MNSENPERC']

e12['DEMBASE_CNT'] = e12[dfl_totals12].apply(min1, axis = 1)
e12['DEMBASE_PCT'] = e12[dfl_percs12].apply(min1, axis = 1)
e12['DEMAV_CNT'] = e12[dfl_totals12].apply(avg1, axis = 1)
e12['DEMAV_PCT'] = e12[dfl_percs12].apply(avg1, axis = 1)
e12['Year'] = '2012'
e12.head(20)



,MNLEGDIST,TOTVOTING,USPRSDFL,USPRSTOTAL,USSENDFL,MNSENDFL,MNSENTOTAL,USSENTOTAL,USREPDFL,USREPTOTAL,...,USSENPERC,USREPPERC,MNLEGPERC,USPRSPERC,MNSENPERC,DEMBASE_CNT,DEMBASE_PCT,DEMAV_CNT,DEMAV_PCT,Year
0,10A,21419,9074,21342,11918,9199,20621,20656,9941,20742,...,0.576975,0.479269,0.569820,0.425171,0.446099,9074,0.425171,10384.2,0.499467,2012
1,10B,22867,10098,22720,13278,10291,21787,21928,11017,22143,...,0.605527,0.497539,0.506927,0.444454,0.472346,10098,0.444454,11154.2,0.505359,2012
2,11A,20674,12732,20555,14584,13941,19779,20056,12759,20264,...,0.727164,0.629639,0.638675,0.619411,0.704838,12610,0.619411,13325.2,0.663946,2012
3,11B,19024,8786,18879,11250,10401,18125,18184,9323,18379,...,0.618676,0.507264,0.512295,0.465385,0.573848,8786,0.465385,9831.2,0.535493,2012
4,12A,22120,10130,21964,13661,8421,21414,21424,13780,21511,...,0.637649,0.640602,0.474851,0.461209,0.393247,8421,0.393247,11216.8,0.521512,2012
5,12B,20613,7423,20499,11855,7380,19695,19981,11463,20017,...,0.593314,0.572663,0.336181,0.362115,0.374714,6611,0.336181,8946.4,0.447798,2012
6,13A,22093,9022,21960,13082,7411,20812,21115,11345,21534,...,0.619560,0.526841,0.407957,0.410838,0.356093,7411,0.356093,9839.4,0.464258,2012
7,13B,21448,8649,21330,12448,7460,20135,20568,10947,20986,...,0.605212,0.521633,0.396230,0.405485,0.370499,7460,0.370499,9490.0,0.459812,2012
8,14A,20325,9610,20227,12280,8522,19156,19404,11403,19864,...,0.632859,0.574054,0.458034,0.475108,0.444874,8522,0.444874,10108.2,0.516986,2012
9,14B,19308,10158,19185,11738,8912,17726,18210,11463,18636,...,0.644591,0.615100,0.562153,0.529476,0.502764,8912,0.502764,10457.6,0.570817,2012


In [19]:
#Rename columns to include the year for all the dataframes.
df_list = [e10, e12, e14]

for df in df_list:
    df.columns = [col + (df['Year'][1]) for col in df.columns]
            

In [20]:
e14.head()

,MNLEGDIST2014,TOTVOTING2014,USSENDFL2014,USSENTOTAL2014,USREPDFL2014,USREPTOTAL2014,MNLEGDFL2014,MNLEGTOTAL2014,MNGOVDFL2014,MNGOVTOTAL2014,...,MNLEGPERC2014,MNGOVPERC2014,MNSOSPERC2014,MNAUDPERC2014,MNAGPERC2014,DEMBASE_CNT2014,DEMBASE_PCT2014,DEMAV_CNT2014,DEMAV_PCT2014,Year2014
0,10A,16357,7197,16273,6294,16236,7539,16200,6727,16222,...,0.465370,0.414684,0.370541,0.436135,0.444731,5869,0.370541,6801.428571,0.423055,2014
1,10B,18015,8640,17899,7705,17842,8523,17757,8096,17820,...,0.479980,0.454321,0.416868,0.490273,0.512577,7246,0.416868,8242.714286,0.466939,2014
2,11A,15575,9658,15474,8275,15455,9493,15226,9127,15413,...,0.623473,0.592162,0.566219,0.584542,0.602256,8275,0.535425,9009.000000,0.589746,2014
3,11B,14306,7061,14181,6501,14059,6488,14058,6493,14136,...,0.461517,0.459324,0.435814,0.501120,0.528018,5992,0.435814,6686.000000,0.478017,2014
4,12A,17117,8709,16946,8864,16880,8065,16828,7964,16849,...,0.479261,0.472669,0.427225,0.479795,0.492886,6980,0.427225,8071.714286,0.484412,2014


In [21]:
emaster = pd.merge(e10, e12, left_on = 'District2010', right_on = 'MNLEGDIST2012', how = 'inner')
#checks = ['LEG2010', 'MNLEGDIST2012', 'MNLEGDIST2014']
#emaster.shape[0]
emaster = pd.merge(emaster, e14, left_on = 'District2010', right_on = 'MNLEGDIST2014', how = 'inner')
#emaster
emaster.shape[0]

134

In [22]:
print(emaster.columns)

Index(['District2010', 'TotVoters2010', 'CONGDFL2010', 'CONGTOT2010', 'MNSENDFL2010', 'MNSENTOT2010', 'MNLEGDFL2010', 'MNLEGTOT2010', 'GOVDFL2010', 'GOVTOT2010', 'SOSDFL2010', 'SOSTOT2010', 'STAUDDFL2010', 'STAUDTOT2010', 'ATGENDFL2010', 'ATGENTOT2010', 'USREPPERC2010', 'MNLEGPERC2010', 'MNGOVPERC2010', 'MNSOSPERC2010', 'MNAUDPERC2010', 'MNAGPERC2010', 'MNSENPERC2010', 'DEMBASE_CNT2010', 'DEMBASE_PCT2010', 'DEMAV_CNT2010', 'DEMAV_PCT2010', 'Year2010', 'MNLEGDIST2012', 'TOTVOTING2012', 'USPRSDFL2012', 'USPRSTOTAL2012', 'USSENDFL2012', 'MNSENDFL2012', 'MNSENTOTAL2012', 'USSENTOTAL2012', 'USREPDFL2012', 'USREPTOTAL2012', 'MNLEGDFL2012', 'MNLEGTOTAL2012', 'USSENPERC2012', 'USREPPERC2012', 'MNLEGPERC2012', 'USPRSPERC2012', 'MNSENPERC2012', 'DEMBASE_CNT2012', 'DEMBASE_PCT2012', 'DEMAV_CNT2012', 'DEMAV_PCT2012', 'Year2012', 'MNLEGDIST2014', 'TOTVOTING2014', 'USSENDFL2014', 'USSENTOTAL2014', 'USREPDFL2014', 'USREPTOTAL2014', 'MNLEGDFL2014', 'MNLEGTOTAL2014', 'MNGOVDFL2014', 'MNGOVTOTAL2014', '

In [34]:


cols2 = ['District2010', 'DEMBASE_PCT2010', 'DEMBASE_PCT2012', 'DEMBASE_PCT2014']
demBaseP = emaster[cols2]
#demBaseP.head()
demBase = demBaseP.mean(axis = 1, numeric_only = True)
demBase = pd.DataFrame(demBase)

cols3 = ['District2010', 'DEMAV_PCT2010', 'DEMAV_PCT2012', 'DEMAV_PCT2014']
demAvP = emaster[cols3]
dpi = demAvP.mean(axis = 1, numeric_only = True)
dpi = pd.DataFrame(dpi)


district_list = emaster['District2010']

stats = pd.concat([district_list, demBase, dpi], axis = 1)
stats.columns = ['District', 'DemBase', 'DPI']
stats.head()

,District,DemBase,DPI
0,10A,0.384084,0.426354
1,10B,0.399135,0.436126
2,11A,0.496667,0.529618
3,11B,0.409999,0.447436
4,12A,0.401893,0.463710


In [50]:
to_merge = ['District2010', 'DEMBASE_CNT2014', 'DEMBASE_CNT2012', 'DEMBASE_CNT2010', 'TOTVOTING2012']

districts = pd.merge(stats, emaster[to_merge], left_on = 'District', right_on = 'District2010', how = "inner")
districts.head()

,District,DemBase,DPI,District2010,DEMBASE_CNT2014,DEMBASE_CNT2012,DEMBASE_CNT2010,TOTVOTING2012
0,10A,0.384084,0.426354,10A,5869,9074,5416,21419
1,10B,0.399135,0.436126,10B,7246,10098,5258,22867
2,11A,0.496667,0.529618,11A,8275,12610,5496,20674
3,11B,0.409999,0.447436,11B,5992,8786,5147,19024
4,12A,0.401893,0.463710,12A,6980,8421,6327,22120


In [51]:
districts['OffYrDemBase'] = (districts['DEMBASE_CNT2014']+districts['DEMBASE_CNT2010'])/2
districts['MidTermDropOffDemBase'] = ((districts['OffYrDemBase']/districts['DEMBASE_CNT2012'])-1).abs()
districts = districts

,District,DemBase,DPI,District2010,DEMBASE_CNT2014,DEMBASE_CNT2012,DEMBASE_CNT2010,TOTVOTING2012,OffYrDemBase,DropOffDemBase
0,10A,0.384084,0.426354,10A,5869,9074,5416,21419,5642.5,0.378168
1,10B,0.399135,0.436126,10B,7246,10098,5258,22867,6252.0,0.380867
2,11A,0.496667,0.529618,11A,8275,12610,5496,20674,6885.5,0.453965
3,11B,0.409999,0.447436,11B,5992,8786,5147,19024,5569.5,0.366094
4,12A,0.401893,0.463710,12A,6980,8421,6327,22120,6653.5,0.209892
